### Optimizing promoter strengths for phix-174
In this script, I will optimize the promoter strengths for each of the three promoter sites in Phix_174. 

#### Main Idea
1. Define a unique normal distribution (**~N(u,o)**) for each of the three promoters (pA, pB, pD). Let the promoter strength be **e^u_value** and step size be the st. deviation **o**. 

2. Define a normal distribution (**~N(0.5,?)**) to describe **lambda**. Here, **lambda** represents the value by which the step size will be scaled.

3. Randomly select a promoter to optimize (pA, pB, pD). Feed promoter values into pinetree & calculate RMSE. If the new error (**NE**) is less than the old error (**OE**), then keep the promoter value. If not, increment the promoter value by the step size (add **e^o**). Keep repeating with randomly selected **o** value until **NE** < **OE**. *Stop after 5 iterations & move onto the next promoter value.*

#### Key Attributes
1. 

#### Terminology/Naming Conventions

##### Import Packages & Set Directories

In [8]:
import pandas as pd
import numpy as np
import pinetree as pt
from sklearn.metrics import mean_squared_error 
base_dir = "/Users/tanviingle/Documents/Wilke/phix174/"

##### Run Pinetree
This function runs the pinetree simulator given parameters

STEPS: 
1. Feed in promoter values
2. Run pinetree simulator
3. Write file as gen.csv --> store as basedir/output/opt_test/gen_#.csv

In [9]:
def run_pt(gen, pA, pB, pD): #Start with only feeding in values for pA

    print(gen, pA, pB, pD)
    print("Defining PhiX-174 genome")
    
    # Create host cell & genome
    CELL_VOLUME = 1.1e-15 # from T7
    model = pt.Model(cell_volume=CELL_VOLUME)
    phage = pt.Genome(name="phix_174", length=5386)
    
    # Read genomic coordinates from csv into dataframe
    genomic_coords = pd.read_csv(base_dir + "output/" + "genomic_coords.csv")
    print(genomic_coords.at[0, "type"])
    
    
    # Add genomic ns (loop through ^ df); hardcode necessary strengths according to preomtimized_model
    ## for length of genomic_coords, add elements
    #for n in genomic_coords:{
    n = 0
    while(n < len(genomic_coords)):
        
        if genomic_coords.at[n, "type"] == "gene": 
            phage.add_gene(name= genomic_coords.at[n, "type"] + "_" + genomic_coords.at[n, "name"], 
                           start= genomic_coords.at[n, "new_start"], 
                           stop= genomic_coords.at[n, "new_end"],
                           rbs_start=genomic_coords.at[n, "new_start"] - 15, 
                           rbs_stop=genomic_coords.at[n, "new_start"] - 1, rbs_strength=1e7) 

        elif genomic_coords.at[n, "type"] == "promoter" and genomic_coords.at[n, "name"] == "A":
            phage.add_promoter(name= genomic_coords.at[n, "type"] + "_" + genomic_coords.at[n, "name"], 
                               start= genomic_coords.at[n, "new_start"], 
                               stop= genomic_coords.at[n, "new_end"],
                               interactions={"ecolipol": pA})

        elif genomic_coords.at[n, "type"] == "promoter" and genomic_coords.at[n, "name"] == "B1":
            phage.add_promoter(name= genomic_coords.at[n, "type"] + "_" + genomic_coords.at[n, "name"], 
                               start= genomic_coords.at[n, "new_start"], 
                               stop= genomic_coords.at[n, "new_end"],
                               interactions={"ecolipol": pB})

        elif genomic_coords.at[n, "type"] == "promoter" and genomic_coords.at[n, "name"] == "D":
            phage.add_promoter(name= genomic_coords.at[n, "type"] + "_" + genomic_coords.at[n, "name"], 
                               start= genomic_coords.at[n, "new_start"], 
                               stop= genomic_coords.at[n, "new_end"],
                               interactions={"ecolipol": pD})

        else:
            print("ignoring pB2")

        n = n+1
    
    print("all genes and promoters added")
    
    # Add terminators manually 
    phage.add_terminator(name="terminator_J", start=2402, stop=2403, # Right before gene F start=2404, stop=3687,
                       efficiency={"ecolipol": 0.3}) # 0.7
    phage.add_terminator(name="terminator_F", start=3796, stop=3797, # Right before gene G start=3798, stop=4325
                     efficiency={"ecolipol": 0.2}) # 0.8
    phage.add_terminator(name="terminator_G", start=4332, stop=4333, # Right before gene H start=4334, stop=5320
                     efficiency={"ecolipol": 0.3}) # 0.6
    phage.add_terminator(name="terminator_H", start=5321, stop=5322, # Right after gene H
                     efficiency={"ecolipol": 0.7}) # 0.3

    print("all terminators added")
    
    # Register genome after promoters/terminators are added
    model.register_genome(phage)
    print("genome is registered")

    # Define interactions
    print("Defining Polymerases & Interactions")
    # Add polymerases & species
    model.add_polymerase(name="ecolipol", speed=35, footprint=35, copy_number=0)
    model.add_species("bound_ecolipol", 2000)  # initialization, 1800
    model.add_species("ecoli_genome", 0)
    model.add_species("ecoli_transcript", 0)
    model.add_reaction(1e7, ["ecolipol", "ecoli_genome"], ["bound_ecolipol"]) # 1e6
    model.add_reaction(0.04, ["bound_ecolipol"], ["ecolipol", "ecoli_genome", "ecoli_transcript"])
    model.add_ribosome(10, 30, 100)
    model.add_species("bound_ribosome", 100)
    model.seed(34)
    
    # Run simulation
    print("running simulation")
    model.simulate(time_limit=1200, time_step=5, output= base_dir + "output/opt_test/gen_"+str(gen)+".tsv") # TODO change limit
    print("Simulation successful!")

In [ ]:
run_pt(gen = 1, 
       pA = np.exp(12.21), 
       pB = np.exp(14.33), 
       pD = np.exp(14.11))

In [3]:
def wrap_pt(gen, pA, pB, pD, j):
    while (j == 0):
        try:
            run_pt(gen, pA, pB, pD)
            j = 1
            print ("good promoter values")
            print (f"\n")
        except:
            print("poor promoeter values chosen, retrying")
            pA = np.exp(np.random.normal(uA, oA, 1))
            pB = np.exp(np.random.normal(uB, oB, 1))
            pD = np.exp(np.random.normal(uD, oD, 1))
            j = 0 
    return pA, pB, pD

##### Calculate Error
This script compares a pinetree run to the experimental data
STEPS:
1. Read pinetree run file
2. Use RMSE to calculate error 
3. Return error

In [10]:
def get_error(file):
    sim = pd.read_csv(file, sep = "\t")
    sim = sim.round({'time': 0})
    sim = sim[sim['time'] == 1200.0]
    sim = sim[sim.species.str.match("gene_")]
    if (len(sim) != 11):
        error = -1
        return(error)
    else :
        sim["norm"] = sim['transcript']/(sim.iloc[0]["transcript"])
        sim["exp"] = [1, 1, 6, 6, 17, 17, 11, 5, 1, 17, 6]
        error = mean_squared_error(sim.exp, sim.norm, squared = False)
        return(error)


##### Define Normal Distributions

In [11]:
gen = 0
error = 1e10
step = 1e2 # some error with reassigning this value, scope? 

uA = 10
oA = 8
pA = np.exp(np.random.normal(uA, oA, 1)[0])

uB = 12
oB = 7
pB = np.exp(np.random.normal(uB, oB, 1)[0])

uD = 12
oD = 9
pD = np.exp(np.random.normal(uD, oD, 1)[0])

j = 0

params = ["pA", "pB", "pD"]
report_df = pd.DataFrame(columns = ['gen', 'pA', 'pB', 'pD', 'error'])

first_run = {'gen': "NA", 'pA': pA, 'pB': pB, 'pD': pD,'error': error} 
report_df = report_df.append(first_run, ignore_index = True)

old_error = error
new_error = error

no_change = 0

#while (old_error >= new_error and no_change <= 10):
while (gen < 100):
    # STEP 1: Randomly select promoter to optimize; if gen == 10, quit. 
    print(gen)
    #if (gen == 30):
    #    break   
        
    while (gen == 0 and j == 0):
        try:
            run_pt(gen, pA, pB, pD)
            j = 1
            gen = 1
            print ("good starting values")
            print(gen, pA, pB, pD)
            print (f"\n")
        except:
            print("poor starting values chosen, retrying")
            pA = np.exp(np.random.normal(uA, oA, 1)[0])
            pB = np.exp(np.random.normal(uB, oB, 1)[0])
            pD = np.exp(np.random.normal(uD, oD, 1)[0])
            j = 0 
    
    if (no_change == 0):
        print('pick random promoter') 
        pX = np.random.choice(params)
        print(pX)
    
          
    # STEP 2: Run pinetree with selected promoter & value
    # Increment pX by oX
    if (pX == "pA"):
        print (pA)
        print("increment pA")
        pA = pA + np.exp(oA)
        print("new pA = ", pA)
        print(pA, pB, pD)
        run_pt(gen = gen, pA = pA, pB = pB, pD = pD)
            
    if (pX == "pB"):
        print (pB)
        print("increment pB")
        pB = pB + np.exp(oB)
        print("new pB = ", pB)
        print(pA, pB, pD)
        run_pt(gen = gen, pA = pA, pB = pB, pD = pD)
        
    if (pX == "pD"):
        print (pD)
        print("increment pD")
        pD = pD + np.exp(oD)
        print("new pD = ", pD)
        print(pA, pB, pD)
        run_pt(gen = gen, pA = pA, pB = pB, pD = pD)
    
    # STEP 3: Calculate Error
    old_error = new_error
    new_error = get_error(file = base_dir + "output/opt_test/gen_" +str(gen)+".tsv")
    
    # STEP 4: Compare Old Error to New Error;
    if ((new_error == -1 or new_error >= old_error)):
        if(no_change >= 4):
            no_change = 0
            continue
        print("entering error loop")
        no_change = no_change + 1
        print("new_error = " + str(new_error) + ", old_error = " + str(old_error) + "\n")
        new_error = old_error
        old_error = report_df.at[gen-1, "error"]

        if (pX == "pA"):
            step = oA*(np.random.normal(0.5, 0.2, 1)[0])
            pA = report_df.at[gen-1, "pA"] + np.exp(step)
            print(step)
            #print(pA, pB, pD)
            new_run = {'gen': gen, 'pA': pA, 'pB': pB, 'pD': pD,'error': new_error} 
            report_df = report_df.append(new_run, ignore_index = True)
            gen = gen + 1
            print(report_df)
            continue

        if (pX == "pB"):
            step = oB*(np.random.normal(0.5, 0.2, 1)[0])
            pB = report_df.at[gen-1, "pB"] + np.exp(step)
            print(step)
            #print(pA, pB, pD)
            new_run = {'gen': gen, 'pA': pA, 'pB': pB, 'pD': pD,'error': new_error} 
            report_df = report_df.append(new_run, ignore_index = True)
            gen = gen + 1
            print(report_df)
            continue

        if (pX == "pD"):
            step = oD*(np.random.normal(0.5, 0.2, 1)[0])
            pD = report_df.at[gen-1, "pD"] + np.exp(step)
            print(step)
            #print(pA, pB, pD)
            new_run = {'gen': gen, 'pA': pA, 'pB': pB, 'pD': pD,'error': new_error} 
            report_df = report_df.append(new_run, ignore_index = True)
            gen = gen + 1
            print(report_df)
            continue

    #print (step)    
    new_run = {'gen': gen, 'pA': pA, 'pB': pB, 'pD': pD,'error': new_error} 
    report_df = report_df.append(new_run, ignore_index = True)
   
    print(pA, pB, pD)
    gen = gen + 1
    no_change = 0
    display(report_df)
    print(f"\n")
        
#display(report_df)
report_df.to_csv(base_dir + "output/opt_test/relative_report_test.csv")
display(report_df)

0
0 2.275162192161076 18026337.822272994 1203.0499400310075
Defining PhiX-174 genome
gene
ignoring pB2
all genes and promoters added
all terminators added
genome is registered
Defining Polymerases & Interactions
running simulation
Simulation successful!
good starting values
1 2.275162192161076 18026337.822272994 1203.0499400310075


pick random promoter
pA
2.275162192161076
increment pA
new pA =  2983.2331492338894
2983.2331492338894 18026337.822272994 1203.0499400310075
1 2983.2331492338894 18026337.822272994 1203.0499400310075
Defining PhiX-174 genome
gene
ignoring pB2
all genes and promoters added
all terminators added
genome is registered
Defining Polymerases & Interactions
running simulation
Simulation successful!
entering error loop
new_error = -1, old_error = 10000000000.0

3.6235410725577077
   gen         pA            pB          pD         error
0   NA   2.275162  1.802634e+07  1203.04994  1.000000e+10
1  1.0  39.745179  1.802634e+07  1203.04994  1.000000e+10
2
39.7451794190

Simulation successful!
entering error loop
new_error = 455.47707256937053, old_error = -1

2.1001906160414356
   gen           pA            pB            pD         error
0   NA     2.275162  1.802634e+07   1203.049940  1.000000e+10
1  1.0    39.745179  1.802634e+07   1203.049940  1.000000e+10
2  2.0   935.311678  1.802634e+07   1203.049940  1.000000e+10
3  3.0   942.080732  1.802634e+07   1203.049940  1.000000e+10
4  4.0   962.593243  1.802634e+07   1203.049940  1.000000e+10
5  5.0  3943.551231  1.802634e+07   1222.005607 -1.000000e+00
6  6.0  3943.551231  1.802634e+07   1256.818109 -1.000000e+00
7  7.0  3943.551231  1.802634e+07   1271.971322 -1.000000e+00
8  8.0  3943.551231  1.802634e+07   3081.110907 -1.000000e+00
9  9.0  3951.718957  1.802634e+07  11184.194834 -1.000000e+00
10
3951.7189571955096
increment pA
new pA =  6932.676944237238
6932.676944237238 18026337.822272994 11184.194834256687
10 6932.676944237238 18026337.822272994 11184.194834256687
Defining PhiX-174 genome
gene


Simulation successful!
entering error loop
new_error = 455.47707256937053, old_error = 455.47707256937053

2.638100834977836
     gen           pA            pB            pD         error
0     NA     2.275162  1.802634e+07   1203.049940  1.000000e+10
1    1.0    39.745179  1.802634e+07   1203.049940  1.000000e+10
2    2.0   935.311678  1.802634e+07   1203.049940  1.000000e+10
3    3.0   942.080732  1.802634e+07   1203.049940  1.000000e+10
4    4.0   962.593243  1.802634e+07   1203.049940  1.000000e+10
5    5.0  3943.551231  1.802634e+07   1222.005607 -1.000000e+00
6    6.0  3943.551231  1.802634e+07   1256.818109 -1.000000e+00
7    7.0  3943.551231  1.802634e+07   1271.971322 -1.000000e+00
8    8.0  3943.551231  1.802634e+07   3081.110907 -1.000000e+00
9    9.0  3951.718957  1.802634e+07  11184.194834 -1.000000e+00
10  10.0  3957.390803  1.802634e+07  11184.194834 -1.000000e+00
11  11.0  3969.865318  1.802634e+07  11184.194834 -1.000000e+00
12  12.0  5978.148817  1.802634e+07  11184.

Simulation successful!
entering error loop
new_error = 455.47707256937053, old_error = 455.47707256937053

1.6281057069249623
     gen           pA            pB            pD         error
0     NA     2.275162  1.802634e+07   1203.049940  1.000000e+10
1    1.0    39.745179  1.802634e+07   1203.049940  1.000000e+10
2    2.0   935.311678  1.802634e+07   1203.049940  1.000000e+10
3    3.0   942.080732  1.802634e+07   1203.049940  1.000000e+10
4    4.0   962.593243  1.802634e+07   1203.049940  1.000000e+10
5    5.0  3943.551231  1.802634e+07   1222.005607 -1.000000e+00
6    6.0  3943.551231  1.802634e+07   1256.818109 -1.000000e+00
7    7.0  3943.551231  1.802634e+07   1271.971322 -1.000000e+00
8    8.0  3943.551231  1.802634e+07   3081.110907 -1.000000e+00
9    9.0  3951.718957  1.802634e+07  11184.194834 -1.000000e+00
10  10.0  3957.390803  1.802634e+07  11184.194834 -1.000000e+00
11  11.0  3969.865318  1.802634e+07  11184.194834 -1.000000e+00
12  12.0  5978.148817  1.802634e+07  11184

Simulation successful!
entering error loop
new_error = 455.47707256937053, old_error = 455.47707256937053

2.413472923530373
     gen           pA            pB            pD         error
0     NA     2.275162  1.802634e+07   1203.049940  1.000000e+10
1    1.0    39.745179  1.802634e+07   1203.049940  1.000000e+10
2    2.0   935.311678  1.802634e+07   1203.049940  1.000000e+10
3    3.0   942.080732  1.802634e+07   1203.049940  1.000000e+10
4    4.0   962.593243  1.802634e+07   1203.049940  1.000000e+10
5    5.0  3943.551231  1.802634e+07   1222.005607 -1.000000e+00
6    6.0  3943.551231  1.802634e+07   1256.818109 -1.000000e+00
7    7.0  3943.551231  1.802634e+07   1271.971322 -1.000000e+00
8    8.0  3943.551231  1.802634e+07   3081.110907 -1.000000e+00
9    9.0  3951.718957  1.802634e+07  11184.194834 -1.000000e+00
10  10.0  3957.390803  1.802634e+07  11184.194834 -1.000000e+00
11  11.0  3969.865318  1.802634e+07  11184.194834 -1.000000e+00
12  12.0  5978.148817  1.802634e+07  11184.

...

Simulation successful!
entering error loop
new_error = 109.78920918990833, old_error = 109.78920918990833

5.1899077680857175
     gen            pA            pB            pD         error
0     NA      2.275162  1.802634e+07   1203.049940  1.000000e+10
1    1.0     39.745179  1.802634e+07   1203.049940  1.000000e+10
2    2.0    935.311678  1.802634e+07   1203.049940  1.000000e+10
3    3.0    942.080732  1.802634e+07   1203.049940  1.000000e+10
4    4.0    962.593243  1.802634e+07   1203.049940  1.000000e+10
..   ...           ...           ...           ...           ...
81  81.0  27935.908459  1.803615e+07  61883.819918  1.097892e+02
82  82.0  27935.908459  1.803724e+07  61889.445321  1.097892e+02
83  83.0  27935.908459  1.803724e+07  61946.755270  1.097892e+02
84  84.0  27935.908459  1.803724e+07  62164.286734  1.097892e+02
85  85.0  27935.908459  1.803724e+07  62343.738735  1.097892e+02

[86 rows x 5 columns]
86
62343.738734858205
increment pD
new pD =  70446.8226624336
27935.908

,gen,pA,pB,pD,error
0,NA,2.275162,1.802634e+07,1203.049940,1.000000e+10
1,1.0,39.745179,1.802634e+07,1203.049940,1.000000e+10
2,2.0,935.311678,1.802634e+07,1203.049940,1.000000e+10
3,3.0,942.080732,1.802634e+07,1203.049940,1.000000e+10
4,4.0,962.593243,1.802634e+07,1203.049940,1.000000e+10
...,...,...,...,...,...
86,86.0,27941.417845,1.803724e+07,70446.822662,1.097892e+02
87,87.0,28060.470226,1.803724e+07,70446.822662,1.097892e+02
88,88.0,28063.897807,1.803724e+07,70446.822662,1.097892e+02
89,89.0,28168.924432,1.803724e+07,70446.822662,1.097892e+02




91
pick random promoter
pB
18037243.214019686
increment pB
new pB =  18038339.847178113
34130.84040634638 18038339.847178113 70446.8226624336
91 34130.84040634638 18038339.847178113 70446.8226624336
Defining PhiX-174 genome
gene
ignoring pB2
all genes and promoters added
all terminators added
genome is registered
Defining Polymerases & Interactions
running simulation
Simulation successful!
entering error loop
new_error = 105.81628053977671, old_error = 105.81628053977671

1.363523821053518
     gen            pA            pB            pD         error
0     NA      2.275162  1.802634e+07   1203.049940  1.000000e+10
1    1.0     39.745179  1.802634e+07   1203.049940  1.000000e+10
2    2.0    935.311678  1.802634e+07   1203.049940  1.000000e+10
3    3.0    942.080732  1.802634e+07   1203.049940  1.000000e+10
4    4.0    962.593243  1.802634e+07   1203.049940  1.000000e+10
..   ...           ...           ...           ...           ...
87  87.0  28060.470226  1.803724e+07  70446.8226

Simulation successful!
entering error loop
new_error = 145.60972467246793, old_error = 105.81628053977671

2.211929619499144
     gen            pA            pB            pD         error
0     NA      2.275162  1.802634e+07   1203.049940  1.000000e+10
1    1.0     39.745179  1.802634e+07   1203.049940  1.000000e+10
2    2.0    935.311678  1.802634e+07   1203.049940  1.000000e+10
3    3.0    942.080732  1.802634e+07   1203.049940  1.000000e+10
4    4.0    962.593243  1.802634e+07   1203.049940  1.000000e+10
..   ...           ...           ...           ...           ...
93  93.0  34130.840406  1.803729e+07  70446.822662  1.058163e+02
94  94.0  34130.840406  1.803730e+07  70446.822662  1.058163e+02
95  95.0  34130.840406  1.803731e+07  70446.822662  1.058163e+02
96  96.0  34130.840406  1.803756e+07  70446.822662  1.058163e+02
97  97.0  34130.840406  1.803757e+07  70446.822662  1.058163e+02

[98 rows x 5 columns]
98
18037569.33654573
increment pB
new pB =  18038665.969704155
34130.840

,gen,pA,pB,pD,error
0,NA,2.275162,1.802634e+07,1203.049940,1.000000e+10
1,1.0,39.745179,1.802634e+07,1203.049940,1.000000e+10
2,2.0,935.311678,1.802634e+07,1203.049940,1.000000e+10
3,3.0,942.080732,1.802634e+07,1203.049940,1.000000e+10
4,4.0,962.593243,1.802634e+07,1203.049940,1.000000e+10
...,...,...,...,...,...
95,95.0,34130.840406,1.803731e+07,70446.822662,1.058163e+02
96,96.0,34130.840406,1.803756e+07,70446.822662,1.058163e+02
97,97.0,34130.840406,1.803757e+07,70446.822662,1.058163e+02
98,98.0,34130.840406,1.803796e+07,70446.822662,1.058163e+02


In [ ]:
'''
CODE from 12/6/2021
gen = 0
error = 1e10
step = 1e2 # some error with reassigning this value, scope? 

uA = 10
oA = 8
pA = np.exp(np.random.normal(uA, oA, 1)[0])

uB = 12
oB = 7
pB = np.exp(np.random.normal(uB, oB, 1)[0])

uD = 12
oD = 9
pD = np.exp(np.random.normal(uD, oD, 1)[0])

j = 0

params = ["pA", "pB", "pD"]
report_df = pd.DataFrame(columns = ['gen', 'pA', 'pB', 'pD', 'error'])

first_run = {'gen': "NA", 'pA': pA, 'pB': pB, 'pD': pD,'error': error} 
report_df = report_df.append(first_run, ignore_index = True)

old_error = error
new_error = error

no_change = 0

#while (old_error >= new_error and no_change <= 10):
while (gen < 11):
    # STEP 1: Randomly select promoter to optimize; if gen == 10, quit. 
    print(gen)
    #if (gen == 30):
    #    break   
        
    while (gen == 0 and j == 0):
        try:
            run_pt(gen, pA, pB, pD)
            j = 1
            gen = 1
            print ("good starting values")
            print(gen, pA, pB, pD)
            print (f"\n")
        except:
            print("poor starting values chosen, retrying")
            pA = np.exp(np.random.normal(uA, oA, 1)[0])
            pB = np.exp(np.random.normal(uB, oB, 1)[0])
            pD = np.exp(np.random.normal(uD, oD, 1)[0])
            j = 0 
    
    if (no_change == 0):
        print('pick random promoter') 
        pX = np.random.choice(params)
        print(pX)
    
          
    # STEP 2: Run pinetree with selected promoter & value
    # Increment pX by oX
    if (pX == "pA"):
        print (pA)
        print("increment pA")
        pA = pA + np.exp(oA)
        print("new pA = ", pA)
        print(pA, pB, pD)
        run_pt(gen = gen, pA = pA, pB = pB, pD = pD)
            
    if (pX == "pB"):
        print (pB)
        print("increment pB")
        pB = pB + np.exp(oB)
        print("new pB = ", pB)
        print(pA, pB, pD)
        run_pt(gen = gen, pA = pA, pB = pB, pD = pD)
        
    if (pX == "pD"):
        print (pD)
        print("increment pD")
        pD = pD + np.exp(oD)
        print("new pD = ", pD)
        print(pA, pB, pD)
        run_pt(gen = gen, pA = pA, pB = pB, pD = pD)
    
    # STEP 3: Calculate Error
    old_error = new_error
    new_error = get_error(file = base_dir + "output/opt_test/gen_" +str(gen)+".tsv")
    
    # STEP 4: Compare Old Error to New Error;
    if ((new_error == -1 or new_error >= old_error)):
        if(no_change >= 4):
            no_change = 0
            continue
        print("entering error loop")
        no_change = no_change + 1
        print("new_error = " + str(new_error) + ", old_error = " + str(old_error) + "\n")
        new_error = old_error
        old_error = report_df.at[gen-1, "error"]

        if (pX == "pA"):
            step = oA*(np.random.normal(0.5, 0.2, 1)[0])
            pA = report_df.at[gen-1, "pA"] + np.exp(step)
            print(step)
            print(pA, pB, pD)
            continue

        if (pX == "pB"):
            step = oB*(np.random.normal(0.5, 0.2, 1)[0])
            pB = report_df.at[gen-1, "pB"] + np.exp(step)
            print(step)
            print(pA, pB, pD)
            continue

        if (pX == "pD"):
            step = oD*(np.random.normal(0.5, 0.2, 1)[0])
            pD = report_df.at[gen-1, "pD"] + np.exp(step)
            print(step)
            print(pA, pB, pD)
            continue

    
    #print (step)    
    new_run = {'gen': gen, 'pA': pA, 'pB': pB, 'pD': pD,'error': new_error} 
    report_df = report_df.append(new_run, ignore_index = True)
    

    print(pA, pB, pD)
    gen = gen + 1
    no_change = 0
    display(report_df)
    print(f"\n")
        
#display(report_df)
report_df.to_csv(base_dir + "output/opt_test/relative_report_test.csv")
display(report_df)

'''

In [ ]:
"""
gen = 0
error = 1e10
step = np.exp(oA)
report_df = pd.DataFrame(columns = ['gen', 'pA', 'error'])

first_run = {'gen': "NA", 'pA': pA, 'error': error} 
report_df = report_df.append(first_run, ignore_index = True)
   
old_error = error
new_error = error

no_change = 0

# Introduce while loop conditional 
while (old_error >= new_error and no_change <= 5):
    run_pt(gen, pA) # output = file
    old_error = new_error
    new_error = get_error(file = base_dir + "output/opt_test/gen_" +str(gen)+".tsv") # output error
    if(new_error > old_error):
        no_change = no_change + 1
        print("new_error = " + str(new_error) + ", old_error = " + str(old_error) + "\n")
        # this means we've stepped too far
        # back up and use smaller step!
        step = oA*np.random.normal(0.5, 0.2, 1)
        new_error = old_error
        old_error = report_df.at[gen-1, "error"]
        pA = report_df.at[gen-1, "pA"] + np.exp(step)
        continue
    new_run = {'gen': gen, 'pA': pA, 'error': new_error} 
    report_df = report_df.append(new_run, ignore_index = True)
    if gen == 6:
        break
    pA = pA + np.exp(step)
    gen = gen + 1
    #display(report_df)
    print(f"\n")

#display(report_df)
report_df.to_csv(base_dir + "output/opt_test/relative_report_test.csv")
display(report_df)

"""

In [ ]:
''''
#### Weird Pinetree Overlap Errors
Some promoter values cause an overlap error in the simulation.
Here I test using try/except to find promoter values to test 
that do not result in this error 
j = 0
k = 0
pA = np.exp(11.9)

while (j==0 and k < 100):
    print(k)
    try:
        print(pA)
        run_pt(gen = 0, pA = pA, pB = 5e6, pD = 2e5)
        j = 1
    except:
        print("Overlap issue")
        print(f"\n")
        j = 0
        pA = pA + np.exp(6*np.random.normal(0.5, 0.2, 1))
        k = k + 1
        
        
uA = 12.21
oA = 2
pA = np.exp(np.random.normal(uA, oA, 1))

uB = 17.7
oB = 3
pB = np.exp(np.random.normal(uB, oB, 1))

uD = 14.5
oD = 2
pD = np.exp(np.random.normal(uD, oD, 1))
''''''

In [ ]:
''''
all_genes = ["gene_A","gene_A*", "gene_B", "gene_C", 
             "gene_D", "gene_E", "gene_F", "gene_G",
             "gene_H", "gene_J", "gene_K"]
sim = pd.read_csv("/Users/tanviingle/Documents/Wilke/phix174/output/opt_test/gen_0.tsv", sep = "\t")
#display(sim)
sim = sim.round({'time': 0})
sim = sim[sim['time'] == 2400.0]
#sim.species.str.match("gene_B")
sim = sim[sim.species.str.match("gene_")]

#print(mean_squared_error([1,2,3,4,5,6,7], [3,4,5,6,7,8,9], squared = False))


for gene in all_genes:
    if (len(sim.loc[sim["species"] == "gene_A"]) == 0):
  
# if len sim loc thingy = 0, add row with gene_X & 0 0 0 
# if len sim loc thingy = 1 move on

len(sim.loc[sim["species"] == "gene_A"])
len(sim.loc[sim["species"] == "gene_B"])

#df.loc[df['column_name'] == some_value]

In [ ]:
#pA = 2e5
#pB = 6.25*pA
#pD = 5.25*pA
#run_pt(gen = 20, pA = 2e5, pB = 5.5e5, pD = 5e5)
run_pt(gen = 1, 
       pA = np.exp(12.21), 
       pB = np.exp(14.33), 
       pD = np.exp(14.22))

run_pt(gen = 2, 
       pA = np.exp(12.21), 
       pB = np.exp(14.20), 
       pD = np.exp(15.55))


#run_pt(gen = 2, 
 #      pA = np.exp(12.21), 
  #     pB = np.exp(14.22), 
   #    pD = np.exp(14.44))
#run_pt(gen = 20, pA = 2e5, pB = 5e6, pD = 2e10) #--> RMSE = 3.58
#run_pt(gen = 0, pA = 2e3, pB = 5e4, pD = 2e5) --> RMSE = 36.5
#gen = 0
#get_error(file = base_dir + "output/opt_test/gen_" +str(gen)+".tsv")